iterate all files related to CAPS



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date 

#### csv file list and endFrame
endFrame: the global data frame
csvFileList: the list of all csv files we must search by
acquire .csv files in the local directory, by type (caps, match, etc) then compile into a global dataframe, containing the entire sequence of data points. This is several months of high frequency intake.

In [2]:
import os

#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("."):
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename)
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

endFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
endFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", endFrame.shape[0])
start = endFrame["time"].min()
end = endFrame["time"].max()
print("start: ", start, " end: ", end)

for new df:  111336
start:  1660221600292  end:  1671726692823


#### get indicators: the Ia dataframe

In [3]:
def getIndicators():
    return pd.read_csv("usIndicatorsFall22-fulls.csv")

#### read csv for the LOB files of type
return a list of all indicator dates, as full readable dates

In [4]:
def getlob(csv):
        print("GETTING LOB FILE FOR: csv named ",csv)
        csv = "lob_caps/" + csv
        df = pd.read_csv(csv)
            # print("\tfor ", type, df.columns)
        return df

convert dates in file names into epoch integers

In [5]:
from dateutil import parser
import datetime
import time

def convertIndicatorDateToEpoch(original):
    # print("original ", original) #Thu Aug 11 4:30 AM 2022
    re =time.strptime(original, "%a %b %d %I:%M %p %Y")
    ep = time.mktime(re)
    # print("TS ", int(ep))   #1660311116579
    return int(ep)

#### get full Indicator Announcement data, and dates
to help join caps csv to the announcement, on demand, return one list with all econ indicator data plus epoch dates, for integration into the epoch-based data set on caps, etc

In [6]:
def getFullIADates():
    indicatorData = getIndicators() #indicators file read, access all indicator data per event/announcement
    result = []
    for row, val in indicatorData.iterrows(): #all indicators
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
        print("FOR INDICATOR, DATED: ",extendedDateTime)
        IaDict = { "dictDF":val, "epochForIA":convertIndicatorDateToEpoch( extendedDateTime)} #get the data for the indicator row, then the epoch date
        result.append( IaDict) #returns df/caps and the date associated    
    return result

#### integration: set up the viz for caps + indicator
iterate the caps by day, then locate an indicator for that day, if one exists, by looking for an indicator whose epoch falls within the range of epoch-based events, that day. 

vizIAwithCaps: the array used to visualise each day of the indicator, which contains: 
* data for the indicator (forecast, consensus, actual, date of) [dictDF, pd.DataFrame]
* epoch of the indicator, to make an axvline in the chart [epochForIA, int]
* capitalization data [capsDF, pd.DataFrame]

In [12]:
vizIAwithCaps = [] #a list of capitalization dataframes, with respecitve indicators

inds = getFullIADates()  #all dates for indicators, as epochs inds[i][dictDF] / epochForIA
for i in range(len(csvFileList)):
    tempDF = getlob(csvFileList[i])         #get lob stats for that day 
    capsWithIndicatorDict = {}
    min = tempDF['time'].min()
    max = tempDF['time'].max()              #respective range of dates, as epoch
    print("for epoch period ", min , " --> ", max)
    for i in range(len(inds)):              #scan indicators for a date that falls inside the min/max range of time
        # if inds[i]['epochForIA'] <= max and inds[i]['epochForIA'] >= min:
        if inds[i]['epochForIA'] in range(min, max):
            print("Economic indicator for caps is ", inds[i]['dictDF']['indicator'])
            print("occuring at time ", inds[i]['epochForIA'])
            capsWithIndicatorDict["indicatorDF"] = inds[i]['dictDF']            #indicator's data frame row (series?)
            capsWithIndicatorDict['indicatorEpoch'] = inds[i]['epochForIA']     #epoch for indicator, during the day of the caps file 
            capsWithIndicatorDict['capsDF'] = tempDF                            #dataframe for the csv, for graphing 
            print(capsWithIndicatorDict)
            vizIAwithCaps.append(capsWithIndicatorDict)

FOR INDICATOR, DATED:  Mon Aug 01 6:00 AM 2022
FOR INDICATOR, DATED:  Tue Aug 02 6:00 AM 2022
FOR INDICATOR, DATED:  Wed Aug 03 6:00 AM 2022
FOR INDICATOR, DATED:  Thu Aug 04 4:30 AM 2022
FOR INDICATOR, DATED:  Fri Aug 05 4:30 AM 2022
FOR INDICATOR, DATED:  Wed Aug 10 4:30 AM 2022
FOR INDICATOR, DATED:  Wed Aug 10 4:30 AM 2022
FOR INDICATOR, DATED:  Thu Aug 11 4:30 AM 2022
FOR INDICATOR, DATED:  Fri Aug 12 6:00 AM 2022
FOR INDICATOR, DATED:  Tue Aug 16 4:30 AM 2022
FOR INDICATOR, DATED:  Wed Aug 17 4:30 AM 2022
FOR INDICATOR, DATED:  Wed Aug 17 10:00 AM 2022
FOR INDICATOR, DATED:  Wed Aug 24 4:30 AM 2022
FOR INDICATOR, DATED:  Fri Aug 26 4:30 AM 2022
FOR INDICATOR, DATED:  Fri Aug 26 4:30 AM 2022
FOR INDICATOR, DATED:  Fri Aug 26 6:00 AM 2022
FOR INDICATOR, DATED:  Tue Aug 30 6:00 AM 2022
FOR INDICATOR, DATED:  Thu Sep 01 6:00 AM 2022
FOR INDICATOR, DATED:  Fri Sep 02 4:30 AM 2022
FOR INDICATOR, DATED:  Tue Sep 06 6:00 AM 2022
FOR INDICATOR, DATED:  Wed Sep 07 4:30 AM 2022
FOR INDICATO

In [8]:
import datetime
def demarcateByAnnounceTime(dfArray): #indicatorDF, indicatorEpoch, capsDF
    
    for i in range(len(dfArray)):
        #if isinstance(dfArray[i], dict):                
        print("start time: ", dfArray[i]["capsDF"]["time"].min())
        print("end time: ", dfArray[i]["capsDF"]["time"].max())
        print("date of data: ", dfArray[i]['indicatorDF']["date"], "\n")
        data1 = dfArray[i]["capsDF"]['mp']
        data2 = dfArray[i]["capsDF"]['bc']
        t= dfArray[i]["capsDF"]['time']
        ai = dfArray[i]['indicatorEpoch']              #["time"]
        fig, ax1 = plt.subplots()
        tt = i + 1

        color = 'tab:red'
        ax1.set_xlabel('time (s)')
        ax1.set_ylabel('price, AVAX', color=color)
        ax1.plot(t, data1, color=color)
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()                        # instantiate a second axes that shares the same x-axis

        color = 'tab:blue'
        ax2.set_ylabel('buy capitalization', color=color)  # we already handled the x-label with ax1
        ax2.plot(t, data2, color=color)
        ax2.tick_params(axis='y', labelcolor=color)

        fig.tight_layout()                      # otherwise the right y-label is slightly clipped
        # e = dfArray[i]["capsDF"]['time'].loc[0]     #datetime.datetime.fromtimestamp(
        # comb = e ," Buy Cap versus Price"       #.strftime('%m-%d')
        titleF = "date of data: " + dfArray[i]['indicatorDF']["date"]
        plt.title(titleF)
        plt.axvline(x=ai, color='black', linestyle='-')
        plt.show()
            
        
        
        
demarcateByAnnounceTime( vizIAwithCaps)  #send the collected 
    

In [9]:
                                # print(type(inds), inds.columns)
def genDatesForIndicators():
    inds = getIndicators() 
    ret = []               
    for key, val in inds.iterrows():
                                    # print(key, val["date"], val["time"])
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
                            # print("FOR INDICATOR, DATED: ",extendedDateTime)
        d = convertIndicatorDateToEpoch(extendedDateTime)
        # print(d)
        ret.append(np.int64(d))
    return ret
        
dateArr = genDatesForIndicators()
for i in range(len(dateArr)):
    print(type(dateArr[i]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'nu

endFrame: collected time series, several months, 111K+ events

dateArr: list of all dates, as epochs for all indicators (dates only)

In [11]:
l = endFrame[(endFrame['time'] <= 1668178800 ) & (endFrame['time'] > 1664578800   )] #1664578800  -->  1668178800
print(l.shape[0])

# for key, val in endFrame.iterrows():
    

0


In [ ]:
# thresh = ((1000 * 60) * 60 ) * 1
# print("threshold minus gate: ", thresh)
# print(endFrame.dtypes)


iterate all anouncement times, associate 1,000 data points before then after each, then calculate

GRAVEYARD 

In [ ]:
# sample = endFrame[endFrame['time'].between(dateArr[i], lowEnd, inclusive="both")]
    # set = endFrame.loc[
    # sample1 = endFrame[(endFrame['time']> lowEnd)]
    # sample = endFrame[(endFrame['time']<= dateArr[i])]# ] #(endFrame['time']<= dateArr[i])]    & (endFrame['time']> lowEnd)
    # s =endFrame[(endFrame['time'] > lowEnd ) & (endFrame['time'] <= dateArr[i] )]
    # sample = endFrame.loc[s]
    # sample = endFrame.loc[set]
    # set = endFrame.query(endFrame['time'] >= lowEnd & endFrame['time'] <= dateArr[i]).index
    # sample = endFrame[set]
    # if int(sample.shape[0]) > 0:
    #  print(s.head()) #ok  
    # print(sample1.shape[0])
        # print(sample["time"].min(), " --> ", sample["time"].max(), "  --> ", sample.shape[0], " items")
    
    
        # sInd = endFrame.iloc[(endFrame['time']-dateArr[i]).abs().argsort()[:1]]#no
    # print("min matching time: ",sInd["time"])  #complete dataframe
    # endFrame[(endFrame['time']==)]
    
    
    
    #get moments 1 hours before the announcement
# print(endFrame["time"].min(), " --> ", endFrame["time"].max(), "  --> ", endFrame.shape[0], " items")

# import math
# for i in range(len(dateArr)): 
    # print("search for : ",dateArr[i])  #int 
# lowEnd = dateArr[i] - thresh  
# print(lowEnd, " --> ", dateArr[i])   #

# print(endFrame[endFrame['time'].isin(dateArr)])

        # print("compare: ", int(val['time']), dateArr[i])
        # if math.isclose(dateArr[i], val['time'], abs_tol = 15000):
        #     print("matched", val['time'])
            # sInd = endFrame.iloc[(endFrame['time']